# Методы анализа неоднородных данных
## Домашнее задание 5 (сдаваемое). Модели с фиксированными эффектами 
## Deadline: 23.59 3 апреля

Задание выполняется на базе данных hw5.dta. Краткое описание данных: 

* county - номер округа штата Северная Каролина
* year - год
* lncrime - натуральный логарифм числа преступлений на человека
* lnpolice - натуральный логарифм числа полицейских на душу населения 
* lndensity - натуральный логарифм плотности населения 


## Задание 1

Рассмотрим парную регрессионную модель для оценивания взаимосвязи логарифма числа преступлений на человека (зависимая переменная) и логарифма числа полицейских на душу населения (предиктор). Так как данные панельные, оценить классическую "объединенную" модель на всей выборке некорректно. Можно воспользоваться уже знакомой Вам моделью с фиксированными эффектами (имеется в виду базовая модель с разными константами для пространственных единиц). Оценивать такую модель на практике мы пока с Вами не учились, но на данном этапе это и не нужно. 
* Ваша задача - получить FE-оценку (FE = fixed-effects) коэффициента при предикторе "логарим числа полицейских на душу населения" на основе взвевешенных оценок коэффициента при данном предикторе, полученных при оценивании регрессионной модели на отдельных подвыборках (в качестве подвыборки в данном случае выступает округ Северной Каролины)
* Опишите, что выступает в качестве "веса" для оценок коэффициентов 
* Объясните алгоритм Ваших действий для получения нужной FE-оценки

In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as sm

In [2]:
df = pd.read_stata('hw5.dta')

df.head()

,county,year,lncrime,lnpolice,lndensity
0,1,81,-3.221757,-6.327340,0.836017
1,1,82,-3.261134,-6.338704,0.845977
2,1,83,-3.496449,-6.300291,0.850920
3,1,84,-3.360270,-6.273361,0.852891
4,1,85,-3.308445,-6.253162,0.860734


In [3]:
df['year'] = pd.Categorical(df.year)
df['county'] = pd.Categorical(df.county)

__FE-estimate__ = weighted sum of coefficients estimated on subsample regressions, where __weights__ is a vector of "group variance" (variance of the predictor in a group) divided by the sum of all group variances. 

In [4]:
samples_coef = []
group_variance = []

# run regressions for counties separetely
for i in df['county'].unique():
    
    # save estimated `lnpolice` coefficients
    samples_coef.append(sm.ols(formula = 'lncrime ~ lnpolice', 
                               data = df.loc[df['county'] == i]).fit().params[1]) # lnpolice coef
    
    # save group variance
    group_variance.append(np.var(np.array(df.loc[df['county'] == i, 'lnpolice'])))

In [5]:
sum(samples_coef * (group_variance/sum(group_variance)))

0.21317326768133676

##### Check the results on estimated FE model

In [6]:
FE_ols = sm.ols(formula = 'lncrime ~ lnpolice + C(county)', data = df).fit()
FE_ols.params[-1] # lnpolice coef

0.2131732781708462

## Задание 2

Проделайте то же самое упражнение, но уже для случая множественной регрессии. Добавьте в Вашу модель переменную "натуральный логарифм плотности населения" в качестве контрольной переменной. 

* Получите FE-оценку коэффициента при предикторе "натуральный логарифм числа преступлений на человека"
* Объясните, что изменилось в алгоритме действия для получения нужной оценки в случае множественной регрессии

To derive this estimate one can use the __Frisch–Waugh–Lovell (FWL) theorem__. 
Theorem (from Greene, 2012): "in the linear least squares regression of vector y on two sets of variables, X1 and
X2, the subvector b2 is the set of coefficients obtained when the residuals from a regression of y on X1 alone are regressed on the set of residuals obtained when each column of X2 is regressed on X1."


Imagine you're more a 'dirty pool' guy and detest fixed effects. Then the FWL-based algorithm of finging a `lnpolice` coefficient will be as follows:

#### The algorithm FWL + FE-estimator (find `lnpolice` coef)

In [7]:
# x_1 = lnpolice
# x_2 = lndensity

reg1 = sm.ols(formula = 'lncrime ~ lndensity', data=df).fit().resid # Qy
reg2 = sm.ols(formula = 'lnpolice ~ lndensity', data=df).fit().resid # QX
# assume that the intersept is a part of x_1, we would regress it only once

b = pd.DataFrame(np.column_stack((np.array(reg1), np.array(reg2))), columns=['res1','res2']) # data

fin = sm.ols('res1 ~ res2', data=b).fit()
fin.params # lnpolice coef

Intercept    2.071260e-15
res2         1.090721e-01
dtype: float64

##### Check it!

In [8]:
sm.ols(formula = 'lncrime ~ lnpolice + lndensity', data=df).fit().params

Intercept   -2.893472
lnpolice     0.109072
lndensity    0.490108
dtype: float64

However, we would prefer to account for the spacial heterogeneity. Let's add some fixed effects in a subtle way!

In [9]:
df['Qy'] = reg1
df['QX'] = reg2

In [10]:
samples_coef_DVA = []
group_var_police = []

# run regressions for counties separetely
for i in df['county'].unique():
    
    # save estimated `lnpolice` coefficients
    final = sm.ols(formula = 'Qy ~ QX', 
                               data = df.loc[df['county'] == i]).fit()
    samples_coef_DVA.append(final.params[1]) # lnpolice coef
    
    # save group variance
    group_var_police.append(np.var(np.array(df.loc[df['county'] == i, 'QX'])))
    #group_var_police.append(np.var(np.array(final.resid)))

In [11]:
sum(np.array(samples_coef_DVA) * (np.array(group_var_police)/sum(np.array(group_var_police))))

0.2074561735377856

##### Check the results on estimated FE model

In [12]:
FE_ols_DVA = sm.ols(formula = 'lncrime ~ lnpolice + lndensity + C(county)', data = df).fit()
FE_ols_DVA.params[-2] # lnpolice coef 

0.21386567548746904

## Задание 3 (самое важное)

Сделайте выводы: 
* Какие пространственные единицы получают больший, а какие - меньший вес при получении FE-оценки? Критически оцените такую процедуру взвешивания. 
* На практике мы, как правило, получаем FE-оценки, запуская автоматический алгоритм. Тем не менее, каким образом исследователю может быть полезно знание о процедуре получения FE-оценки? Порассуждайте о том, какое смещение мы получаем в результате оценивания FE-модели вместо оценивания классической регрессионной модели на отдельных подвыборках.   


Делаем выводы:

* Больший вес получают пространственные единицы с большей условной внутригрупповой вариацией по остаткам 'lnpolice'. Большая вариация несет больше информации, поэтому логично, что она должна сильнее влиять на итоговую оценку коэффициента -- вариацию в данных мы и хотим объяснить. 
* FE-estimator приведет к смещению в случае отсутсвия внутригрупповой вариации по показателю -- коэффициенты при таких группах обнулятся и не будут влиять на итоговый коэффициент при предикторе в FE-модели. То есть перед выбором в пользу и оценкой FE модели стоит посмотреть на внутрегрупповые вариации по показателю == посмотреть на особенность данных, построить графики с распределениями переменных 